## Tutorial to demonstrate capability of Autoscript to handle various hardware modules
- hardware
    - vacuum
    - stage
    - beam
    - detectors - HAADF, ceta
    - aberration coefficients

#### Contributor(s): Utkarsh Pratiush <utkarshp1161@gmail.com> - 23rd May 2025
#### edited - 
   

## 0. Setup imports

In [ ]:
from stemOrchestrator.logging_config   import setup_logging
data_folder  = "."
out_path = data_folder
setup_logging(out_path=out_path) 

In [ ]:
from stemOrchestrator.acquisition import TFacquisition, DMacquisition
from stemOrchestrator.simulation import DMtwin
from stemOrchestrator.process import HAADF_tiff_to_png, tiff_to_png
from autoscript_tem_microscope_client import TemMicroscopeClient
import matplotlib.pyplot as plt
import logging
plot = plt
from typing import Dict

## 01. setup the microscope configuration

In [ ]:
import os
import json
from pathlib import Path

ip = os.getenv("MICROSCOPE_IP")
port = os.getenv("MICROSCOPE_PORT")

if not ip or not port:
    secret_path = Path("../../config_secret.json")
    if secret_path.exists():
        with open(secret_path, "r") as f:
            secret = json.load(f)
            ip = ip or secret.get("ip_TF")
            port = port or secret.get("port_TF")



config = {
    "ip": ip,
    "port": port,
    "haadf_exposure": 40e-8,  # micro-seconds per pixel
    "haadf_resolution": 512, # square
    "out_path": "."
}

## 02. Initialize microscope

In [ ]:

ip = config["ip"]
port = config["port"]
haadf_exposure = config["haadf_exposure"]
out_path = config["out_path"]
haadf_resolution = config["haadf_resolution"]



microscope = TemMicroscopeClient()
microscope.connect(ip, port = port)# 7521 on velox  computer
# microscope.connect( port = port)# 7521 on velox  computer

# query state:

tf_acquisition = TFacquisition(microscope=microscope)

# put beam shift to 0,0
# tf_acquisition.move_beam_shift_positon([0, 0])


In [ ]:
ip

In [ ]:
tf_acquisition.ceta_cam.insertion_state

In [ ]:
tf_acquisition.microscope.specimen.piezo_stage.is_enabled

In [ ]:
tf_acquisition.query_state_of_microscope()

In [ ]:
tf_acquisition.ceta_cam.retract()

In [ ]:
tf_acquisition.query_vacuum_state()


In [ ]:
tf_acquisition.query_vacuum_valves()

In [ ]:
tf_acquisition.open_vacuum_valves()
# tf_acquisition.close_vacuum_valves()

In [ ]:
tf_acquisition.query_is_beam_blanked()

In [ ]:
tf_acquisition.unblank_beam()

In [ ]:
# query mode of microscope
tf_acquisition.microscope.optics.optical_mode
# tf_acquisition.microscope.optics.optical_mode = "STEM"


In [ ]:
# query screen state
tf_acquisition.query_screen_postion()

In [ ]:
tf_acquisition.insert_screen()

In [ ]:
tf_acquisition.query_screen_current()

In [ ]:
tf_acquisition.query_FOV()

In [ ]:
# insert screen

In [ ]:
# measure screen current

In [ ]:
# measure screen current

In [ ]:
# query field of view
# if needed # set it # 3.279404126033114e-08, '2574.2481486146535Kx', '2.5742481486146533Mx'

In [ ]:
# where is the stage?
tf_acquisition.query_stage_position()

In [ ]:
#  query beam shift
tf_acquisition.query_beam_shift_position()

In [ ]:
# acquire haadf
haadf_image, haadf_image_data, haadf_tiff_name, pixel_size_tuple = tf_acquisition.acquire_haadf(exposure = 10e-6, resolution=1024, return_adorned_object=True, return_pixel_size=True)
# tf_acquisition.haadf_det.retract()    
HAADF_tiff_to_png(haadf_tiff_name)

In [ ]:
tf_acquisition.vt_plot_image(haadf_image)

In [ ]:
# acquire haadf
bf_image, bf_image_data, bf_tiff_name, pixel_size_tuple = tf_acquisition.acquire_bf(exposure = 10e-6, resolution=512, return_adorned_object=True, return_pixel_size=True)
# tf_acquisition.haadf_det.retract()    
HAADF_tiff_to_png(bf_tiff_name)

In [ ]:
haadf_image, bf_image, haadf_data, bf_data, haadf_filename, bf_filename, pixel_size_tuple = tf_acquisition.acquire_haadf_bf(exposure = 10e-6, resolution=512, return_adorned_object=True, return_pixel_size=True)
tf_acquisition.vt_plot_images([haadf_image, bf_image])

In [ ]:
tf_acquisition.vt_add_scale_bar(haadf_image)

In [ ]:
tf_acquisition.optistem_c1()

In [ ]:
# acquire haadf
haadf_np_array, haadf_tiff_name = tf_acquisition.acquire_haadf(exposure = 10e-6, resolution=512)
# tf_acquisition.haadf_det.retract()    
HAADF_tiff_to_png(haadf_tiff_name)

In [ ]:
tf_acquisition.optistem_b2_a2()

In [ ]:
# query stage
tf_acquisition.query_stage_position()


In [ ]:
# move stage 

tf_acquisition.move_stage_translation_absolute(x = 1.070307e-05 , y = 5.888724e-06, z =-110.776122e-06)
# tf_acquisition.move_stage_translation_relative(x =100.701555e-06, y=+100.701555e-06, z = 0.0)
# Gerd: 10UM IN X AND -4 UM IN Y and z is around -90um

In [ ]:
tf_acquisition.microscope.specimen.stage.get_axis_limits(axis="x")

In [ ]:
tf_acquisition.microscope.specimen.stage.start_jogging()

In [ ]:
tf_acquisition.microscope.specimen.stage.stop_jogging()

In [ ]:
from PIL import Image
import numpy as np
import math

from autoscript_tem_microscope_client import TemMicroscopeClient
from autoscript_tem_microscope_client.enumerations import *
from autoscript_tem_microscope_client.structures import *



microscope = TemMicroscopeClient()
# ip = "10.46.218.253"#--> velo computer
# ip = "10.46.217.242"# gatan computer
# microscope.connect(ip, 9095)# 7521 on gatan computer
microscope = tf_acquisition.microscope
ImagesX = 4
ImagesY = 3
Overlap = 0.1

fov = microscope.optics.scan_field_of_view
shift = fov * (1 - Overlap)
OffsetX = math.floor(ImagesX / 2)
OffsetY = math.floor(ImagesY / 2)
InitialStagePosition = microscope.specimen.stage.position
images = []

for y in range(ImagesY):
    for x in range(ImagesX):
        shiftX = (x - OffsetX) * shift
        shiftY = (y - OffsetY) * shift
        # y = pointing to the right
        relativeShift = StagePosition(x=shiftY, y=shiftX)
        position = InitialStagePosition + relativeShift
        microscope.specimen.stage.absolute_move(position)
        image = microscope.acquisition.acquire_stem_image("HAADF", 512, 1E-6)
        images.append(image)

microscope.specimen.stage.absolute_move(InitialStagePosition)

# Display the images in a grid
from matplotlib import pyplot as plt
n = len(images)
fig, plots = plt.subplots(ImagesY, ImagesX)

for row in range(ImagesY):
    for column in range(ImagesX):
        plot = plots[row, column]
        plot.axis('off')
        i = column + row * ImagesX
        plot.imshow(images[i].data, cmap='gray')

plt.show()

In [ ]:
tf_acquisition.move_stage_translation_relative(x =0.1e-6, y=0.5e-6, z = 0.0)

## 03. Aberration's

In [ ]:
import os
import json
from pathlib import Path

# Initialize to None
ip = os.getenv("MICROSCOPE_IP")
port = os.getenv("MICROSCOPE_PORT")


if not ip or not port:
    secret_path = Path("../../config_secret.json")
    if secret_path.exists():
        with open(secret_path, "r") as f:
            secret = json.load(f)
            ip = ip or secret.get("ip_CEOS_sim")
            port = port or secret.get("port_CEOS_sim")


if not ip:
    ip = input("Enter microscope IP: ")
if not port:
    port = input("Enter microscope Port: ")

try:
    port = int(port)
except ValueError:
    raise ValueError("Port must be an integer")


In [ ]:
from stemOrchestrator.acquisition import CEOSacquisition, CEOSacquisitionTCP
# ceos_acquisition = CEOSacquisition(host=ip, port=port)
ceos_acquisition = CEOSacquisitionTCP(host=ip, port=port)


In [ ]:
tableau_result = ceos_acquisition.run_tableau(tab_type="Enhanced", angle=34)# use "Fast", "Standard", "Enhanced" --angle is in mili-radian 9,18,34

In [ ]:
tableau_result 

In [ ]:
tableau_result 

In [ ]:
from autoscript_tem_microscope_client.structures import RunOptiStemSettings, RunStemAutoFocusSettings,RunAutoComaCorrectionSettings, Point, StagePosition, AdornedImage
from autoscript_tem_microscope_client.enumerations import DetectorType, CameraType, OptiStemMethod, OpticalMode

# CameraType.FLUCAM
camera_type = CameraType.BM_CETA
settings = RunAutoComaCorrectionSettings(CameraType.BM_CETA, 1024, 2, 5, tile_size=1024)

In [ ]:
tf_acquisition.optistem_c1()

## Useful Links
- [link to offline TF-Autoscript](https://www.thermofisher.com/us/en/home/electron-microscopy/products/software-em-3d-vis/software-updates.html#autoscript)
- [Autoscript examples](https://www.fei-software-center.com/tem-apps/example-scripts/)

## testing stuff

In [ ]:
tf_acquisition.microscope.detectors.camera_detectors

In [ ]:
tf_acquisition.microscope.detectors.scanning_detectors


In [ ]:
tf_acquisition.microscope.detectors.eds_detectors

In [ ]:
# Get SuperX detector object 
from autoscript_tem_microscope_client.enumerations import DetectorType, CameraType, OptiStemMethod, OpticalMode, EdsDetectorType
detector = tf_acquisition.microscope.detectors.get_eds_detector(EdsDetectorType.ULTRA_X)

# Print detector name 
print(detector.name)

# Print detector display name 


# Inspect detector state and properties 
print(detector.insertion_state)
print(detector.is_insertable)
print(detector.bin_counts)
print(detector.dispersions)
print(detector.shaping_times)

In [ ]:
microscope.source.field_emission_gun.is_flashing_advised


In [ ]:
microscope.source.field_emission_gun.flash()

## get EDS 

In [ ]:
eds_detector.dispersions[-1]# 

In [ ]:
eds_detector.dispersions

In [ ]:
# Acquire single EDS spectrum and plot it
import autoscript_tem_toolkit.vision as vision_toolkit
from autoscript_tem_microscope_client.structures import RunOptiStemSettings, RunStemAutoFocusSettings, Point, StagePosition, AdornedImage, EdsAcquisitionSettings, AdornedSpectrum,  StemAcquisitionSettings, StageVelocity, EdsSpectrumImageSettings
from autoscript_tem_microscope_client.enumerations import DetectorType, CameraType, OptiStemMethod, OpticalMode, EdsDetectorType, ExposureTimeType

# Get the first EDS detector
eds_detector_name = microscope.detectors.eds_detectors[0]
eds_detector = microscope.detectors.get_eds_detector(eds_detector_name)

# Configure the acquisition
settings = EdsAcquisitionSettings()
settings.eds_detector = eds_detector_name
settings.dispersion = eds_detector.dispersions[-1]
settings.shaping_time = eds_detector.shaping_times[-1]
settings.exposure_time = 3e-3
settings.exposure_time_type = ExposureTimeType.LIVE_TIME

# Run the spectrum acquisition
spectrum = microscope.analysis.eds.acquire_spectrum(settings)

# Plot the spectrum
vision_toolkit.plot_spectrum(spectrum)

In [ ]:
spectrum.metadata.metadata_as_xml

In [ ]:
import xmltodict
import json

xml_string = spectrum.metadata.metadata_as_xml

# Parse XML
metadata = xmltodict.parse(xml_string)
metadata = json.loads(json.dumps(metadata))  # OrderedDict → dict

out = {}

# Acceleration voltage (from Optics)
out["AccelerationVoltage"] = float(
    metadata["Metadata"]["Optics"]["AccelerationVoltage"]
)

# Dispersion + Offset for each analytical detector
out["AnalyticalDetectors"] = {}
for det in metadata["Metadata"]["Detectors"]["AnalyticalDetector"]:
    name = det["DetectorName"]
    out["AnalyticalDetectors"][name] = {
        "Dispersion": float(det.get("Dispersion", 0)),
        "OffsetEnergy": float(det.get("OffsetEnergy", 0)),
    }

print(json.dumps(out, indent=2))



In [ ]:
out = {
  "AccelerationVoltage": {
    "value": float(metadata["Metadata"]["Optics"]["AccelerationVoltage"]),
    "unit": "V"
  },
  "AnalyticalDetectors": {}
}

for det in metadata["Metadata"]["Detectors"]["AnalyticalDetector"]:
    name = det["DetectorName"]
    out["AnalyticalDetectors"][name] = {
        "Dispersion": {
            "value": float(det.get("Dispersion", 0)),
            "unit": "eV/channel"
        },
        "OffsetEnergy": {
            "value": float(det.get("OffsetEnergy", 0)),
            "unit": "eV"
        }
    }

out

In [ ]:
spectrum.data

In [ ]:
spectrum.metadata.

In [ ]:
import autoscript_tem_toolkit.vision as vision_toolkit
from autoscript_tem_microscope_client.structures import RunOptiStemSettings, RunStemAutoFocusSettings, Point, StagePosition, AdornedImage, EdsAcquisitionSettings, AdornedSpectrum,  StemAcquisitionSettings, StageVelocity, EdsSpectrumImageSettings, Region
from autoscript_tem_microscope_client.enumerations import DetectorType, CameraType, OptiStemMethod, OpticalMode, EdsDetectorType, ExposureTimeType, ImageSize, RegionCoordinateSystem

In [ ]:
# Get the first EDS detector
eds_detector_name = microscope.detectors.eds_detectors[0]
eds_detector = microscope.detectors.get_eds_detector(eds_detector_name)

# Configure the acquisition
settings = EdsSpectrumImageSettings(eds_detector=eds_detector.name,
                                    dispersion=eds_detector.dispersions[0],
                                    shaping_time=eds_detector.shaping_times[0],
                                    dwell_time=4e-6,
                                    size=ImageSize.PRESET_256,
                                    scanning_detectors=[DetectorType.HAADF])

# Run the scan
eds_spectrum_image = microscope.analysis.eds.acquire_spectrum_image(settings)

# Save the spectrum image to disk in EMD format
eds_spectrum_image.save(r"spectrum_image.emd")

In [ ]:
eds_spectrum_image.

In [ ]:
vision_toolkit.plot_spectra(eds_spectrum_image)

In [ ]:
#Acquire EDS spectrum image with a rectangular region
# Get the first EDS detector
eds_detector_name = microscope.detectors.eds_detectors[0]
eds_detector = microscope.detectors.get_eds_detector(eds_detector_name)

# Configure the acquisition
settings = EdsSpectrumImageSettings(eds_detector=eds_detector.name,
                                    dispersion=eds_detector.dispersions[0],
                                    shaping_time=eds_detector.shaping_times[0],
                                    dwell_time=4e-6,
                                    size=ImageSize.PRESET_1024,
                                    region=Region(coordinate_system=RegionCoordinateSystem.RELATIVE,
                                                rectangle=[0, 0, 0.5, 0.5]),
                                    scanning_detectors=[DetectorType.HAADF])

# Run the scan
eds_spectrum_image = microscope.analysis.eds.acquire_spectrum_image(settings)

# Save the spectrum image to disk in EMD format
eds_spectrum_image.save(r"spectrum_image_rectangle.emd")

## read edx- pytemlib and hyperspy try